# Avoid Rewriting Documents

In [2]:
import datetime
import json
import time
import os
import datetime
import json
import time
from openai import AzureOpenAI
from dotenv import load_dotenv
import json
import copy
import textwrap

# Load environment variables
load_dotenv()

def aoai_call(system_message,prompt,model):
    client = AzureOpenAI(
        api_version=os.getenv("API_VERSION"),
        azure_endpoint=os.getenv("AZURE_ENDPOINT"),
        api_key=os.getenv("API_KEY")
    )

    start_time = time.time()

    completion = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": prompt},
        ],
    )

    end_time = time.time()
    e2e_time = end_time - start_time

    result=json.loads(completion.model_dump_json(indent=2))
    prompt_tokens=result["usage"]["prompt_tokens"]
    completion_tokens=result["usage"]["completion_tokens"]
    completion_text=result["choices"][0]["message"]["content"]

    return result,prompt_tokens,completion_tokens,completion_text,e2e_time

model=os.getenv("MODELGPT432k")

# Read essay from a text file
with open('document_with_errors.txt', 'r') as f:
    document_with_errors = f.read()


# Use Case: Rewrite document to fix grammar and spelling errors


### A: LLM used to rewrite document and implement corrections

Time taken: 315s

In [52]:
import json
import textwrap

# Initialize variables
total_prompt_tokens = 0
total_completion_tokens = 0
total_time = 0
rewritten_document = ""
list_of_errors_to_correct = []
completion_text_list=[]

# Split the text into chunks of words
chunks = textwrap.wrap(document_with_errors, 1000)
for i, chunk in enumerate(chunks):
    system_message="""
    You are a helpful AI assistant.
    """
    prompt=f"""
    Document to correct:
    {chunk}

    """
    prompt=prompt+"""
    The output should be a JSON object. Only return the JSON object, with no comments or additional text.
    Use this structure:
    {
        "rewritten_document": "The document with the errors corrected.",
        "list_of_errors_to_correct": [
            {
                "explanation_of_violation": "Think step-by-step about identifying potential spelling errors or grammar issues. Consider all errors together and consider the whole sentence of text before applying a rule.",
                "violating_text": "If there is a error fill this with the errors text sub-string, consider the full sentence and context of the incorrect text before filling this in",
                "fixed_text": "Think step by step about the error and then fill this with the fixed version of the text, don't apply any other fixes apart from the error if there is one"
            }
        ]
    }
    JSON:
    """

    result,prompt_tokens,completion_tokens,completion_text,e2e_time=aoai_call(system_message,prompt,model)
    total_prompt_tokens += prompt_tokens
    total_completion_tokens += completion_tokens
    total_time += e2e_time
    completion_text_list.append(completion_text)

# Print totals
print(f"Total Prompt Tokens: {total_prompt_tokens}")
print(f"Total Completion Tokens: {total_completion_tokens}")
print(f"Total Time Taken: {total_time:.2f} seconds")
print(f"Rewritten Document: {rewritten_document}")
print(f"List of Errors to Correct: {list_of_errors_to_correct}")

Total Prompt Tokens: 3292
Total Completion Tokens: 3725
Total Cost: $0.6445
Total Time Taken: 315.50 seconds
Rewritten Document: 
List of Errors to Correct: []


In [53]:
import json

# Assuming completion_text_list is your list of dictionaries
combined_list = {"list_of_errors_to_correct":[],"rewritten_document":""}

for completion_text in completion_text_list:
    json_string = completion_text.replace('\n', '')
    data = json.loads(json_string)
    if not data["list_of_errors_to_correct"]:
        pass
    else:
        combined_list["list_of_errors_to_correct"].append(data["list_of_errors_to_correct"])
        combined_list["rewritten_document"]=combined_list["rewritten_document"]+data["rewritten_document"]

document_with_corrections=combined_list["rewritten_document"]
print(document_with_corrections)

Product Launch: Global Expansion of a Leading RTS Video Game. Executive Summary: The launch of our flagship real-time strategy (RTS) video game has set a new benchmark in the North American markets. The exceptional response has resulted in sales that exceeded our initial projections by 1.5 times. As we expanded globally, the international sales mirrored this success, testifying to the universal appeal of the game. However, despite the impressive sales, the feedback regarding the game's micro-transaction system has been less than stellar, indicating an area needing our immediate attention. This report digs deeper into the product's description, its performance in different markets, user feedback, and our plans for future improvements. Product Description: Our RTS game provides a unique blend of immersive gameplay, a dynamic environment, and strategic planning. It's designed to appeal to players in an...Tactical decisions and resource management weave an intricate web in the game. The br

### B: LLM used to identify errors, and python used to implement corrections

Time taken: 19s

In [5]:
import json
import textwrap

# Initialize variables
total_prompt_tokens = 0
total_completion_tokens = 0
total_time = 0
rewritten_document = ""
list_of_errors_to_correct = []
completion_text_list=[]

# Split the text into chunks of words
chunks = textwrap.wrap(document_with_errors, 1000)
for i, chunk in enumerate(chunks):
    system_message="""
    You are a helpful AI assistant.
    """
    prompt=f"""
    Document to correct:
    {chunk}

    """
    prompt=prompt+"""
    The output should be a JSON object. Only return the JSON object, with no comments or additional text.
    Use this structure:
    {
        "list_of_errors_to_correct": [
            {
                "explanation_of_violation": "Think step-by-step about identifying potential spelling errors or grammar issues. Consider all errors together and consider the whole sentence of text before applying a rule.",
                "violating_text": "If there is a error fill this with the errors text sub-string, consider the full sentence and context of the incorrect text before filling this in",
                "fixed_text": "Think step by step about the error and then fill this with the fixed version of the text, don't apply any other fixes apart from the error if there is one"
            }
        ]
    }
    JSON:
    """

    result,prompt_tokens,completion_tokens,completion_text,e2e_time=aoai_call(system_message,prompt,model)
    total_prompt_tokens += prompt_tokens
    total_completion_tokens += completion_tokens
    total_time += e2e_time
    completion_text_list.append(completion_text)

# Print totals
print(f"Total Prompt Tokens: {total_prompt_tokens}")
print(f"Total Completion Tokens: {total_completion_tokens}")
print(f"Total Time Taken: {total_time:.2f} seconds")
print(f"Rewritten Document: {rewritten_document}")
print(f"List of Errors to Correct: {list_of_errors_to_correct}")

Total Prompt Tokens: 3059
Total Completion Tokens: 112
Total Cost: $0.1970
Total Time Taken: 19.27 seconds
Rewritten Document: 
List of Errors to Correct: []


In [6]:
import json

# Assuming completion_text_list is your list of dictionaries
combined_list = {"list_of_errors_to_correct":[]}

for completion_text in completion_text_list:
    json_string = completion_text.replace('\n', '')
    data = json.loads(json_string)
    if not data["list_of_errors_to_correct"]:
        pass
    else:
        combined_list["list_of_errors_to_correct"].append(data["list_of_errors_to_correct"])

# Assuming document_with_errors is your text document
for error_list in combined_list["list_of_errors_to_correct"]:
    for error_dict in error_list:
        document_with_errors = document_with_errors.replace(error_dict['violating_text'], error_dict['fixed_text'])
document_with_corrections=document_with_errors
print(document_with_corrections)

Product Launch: Global Expansion of a Leading RTS Video Game

Executive Summary:

The launch of our flagship real time strategy (RTS) video game has set a new benchmark in the North American markets. The enthusiasm and response received have been exceptional, resulting in sales figures that outstripped our initial projections by 1.5 times. As we expanded into global markets, the international sales figures mirrored this success, testifying to the game's universal appeal. However, despite the impressive sales numbers, user feedback regarding the game's micro-transaction system has been less than stellar, indicating an area that requires our immediate attention. This report will delve deeper into the product's description, its performance in different markets, user feedback and our plans for future improvements.

Product Description:

Our RTS game offers a unique blend of immersive gameplay, a dynamic environment, and strategic planning. The game has been designed to engage players in an